In [5]:
print(x_train.shape, y_train.shape)

(3601, 500, 1) (3601,)


In [1]:
import math
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import plotly.graph_objects as go

#load data
asset_pchange_list = []
asset_price_list = []
asset_name_list = []

df = pd.read_excel('Data.xlsx', sheet_name=4)
date_list = np.flip(df.iloc[:,0].to_numpy().astype('datetime64[D]'))
num_of_asset = len(df.columns)
for i in range(1, num_of_asset):
    asset_array = df.iloc[:,i].to_numpy()
    mask = ~np.isnan(asset_array)
    asset_array_no_na = asset_array[mask]
    asset_price_list.append(np.flip(asset_array_no_na))
    asset_name_list.append(df.columns[i])

p_date_list = date_list[1:]
for i in range(len(asset_price_list)):
    asset_price = asset_price_list[i]
    print(asset_name_list[i], i)
    if i < 12 or (len(asset_price_list)>20 and i<19):
    
        try:
            diff_list = np.diff(asset_price) / asset_price[:-1]
        except:
            print(i)
    else:
        diff_list = np.diff(asset_price)
    asset_pchange_list.append(diff_list)
spx_price = np.flip(df.filter(['SPX index']).values)
date_list = date_list

#Train test split
training_data_len = math.ceil(len(spx_price)* .8)
training_data_len
#Scale data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(spx_price)
scaled_data

#Train test split
train_data = scaled_data[0:training_data_len, :]
x_train = []
y_train = []
look_back_window = 14

for i in range(look_back_window, len(train_data)):
    x_train.append(train_data[i-look_back_window:i, 0])
    y_train.append(train_data[i, 0])
    if i <=15:
        print(x_train)
        print(y_train)
#Convert to np array
x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
test_data = scaled_data[training_data_len-look_back_window:, :]

x_test = []
y_test = spx_price[training_data_len:, :]

for i in range(look_back_window, len(test_data)):
    x_test.append(test_data[i-look_back_window:i, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

print(y_test.shape)

SPX index 0
[array([-0.48081145, -0.45760021, -0.48091928, -0.56051187, -0.49757295,
       -0.53408556, -0.57430334, -0.58678134, -0.51247205, -0.52629293,
       -0.45826675, -0.39652368, -0.42483198, -0.40741375])]
[-0.5640406014624]
[array([-0.48081145, -0.45760021, -0.48091928, -0.56051187, -0.49757295,
       -0.53408556, -0.57430334, -0.58678134, -0.51247205, -0.52629293,
       -0.45826675, -0.39652368, -0.42483198, -0.40741375]), array([-0.45760021, -0.48091928, -0.56051187, -0.49757295, -0.53408556,
       -0.57430334, -0.58678134, -0.51247205, -0.52629293, -0.45826675,
       -0.39652368, -0.42483198, -0.40741375, -0.5640406 ])]
[-0.5640406014624, -0.48760426896037795]
(246, 1)


In [29]:
from tensorflow import keras
from tensorflow.keras import layers
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1)(x)
    return keras.Model(inputs, outputs)

In [30]:
input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(optimizer='adam', loss="mean_squared_error")
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=64,
    callbacks=callbacks,
)

model.evaluate(x_test, y_test, verbose=1)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 14, 1)]      0           []                               
                                                                                                  
 layer_normalization_16 (LayerN  (None, 14, 1)       2           ['input_3[0][0]']                
 ormalization)                                                                                    
                                                                                                  
 multi_head_attention_8 (MultiH  (None, 14, 1)       7169        ['layer_normalization_16[0][0]', 
 eadAttention)                                                    'layer_normalization_16[0][0]'] 
                                                                                            

13870526.0

In [32]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

8/8 [==============================] - 1s 29ms/step


In [33]:
rmse = np.sqrt(np.mean(predictions - y_test)**2)
print(rmse)

15.749463009717982


In [34]:
train = np.squeeze(spx_price[:training_data_len])
valid = np.squeeze(spx_price[training_data_len:])
predictions = np.squeeze(predictions)
train_date_list = date_list[:training_data_len]
valid_date_list = date_list[training_data_len:]
valid_date_list2 = date_list[training_data_len:]
fig = go.Figure()
fig.add_trace(go.Scatter(x=train_date_list, y=train, name='train',  mode='lines'))
fig.add_trace(go.Scatter(x=valid_date_list, y=valid, name='valid',  mode='lines'))
fig.add_trace(go.Scatter(x=valid_date_list2, y=predictions, name='predict',  mode='lines'))
fig.show()